## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

## Get the dataset

In [2]:
data = pd.read_csv('./../data/regions_cleaned.csv')
sweeper_data = pd.read_csv('./../data/2019-09-28-basel-sweeper-routes.csv')
sweeper_data.head()

,osm_id,cci_id,date
0,983964215,251,2019-04-23 11:46:11
1,983964215,251,2019-04-24 11:08:58
2,983964215,251,2019-04-25 7:22:24
3,983964215,251,2019-04-25 6:48:11
4,983964215,251,2019-04-29 11:06:12


### The different between cleaning last time in days

In [3]:
def nearestDate(base, dates):
    nearness = { abs(base.timestamp() - date.timestamp()) : date for date in dates }
    return nearness[min(nearness.keys())]

In [4]:
import dateutil

for index, row in sweeper_data.iterrows():
    temp_df = data[data['osm_id'].astype(str).str.contains(str(row['osm_id']))]
    temp_temp_df = temp_df[temp_df['cci_id'].astype(str).str.contains(str(row['cci_id']))]
    
    if not temp_temp_df.empty:
        date_time_to_check = dateutil.parser.parse(row['date'])
        # print(date_time_to_check)
        date_time_list = []
        for ind, date_val in temp_temp_df['date'].iteritems():
            date_val_ = dateutil.parser.parse(date_val)
            if date_val_>date_time_to_check: # We are only interested in checking the nearest date which came after clearning
                date_time_list.append(date_val_)
        
        if date_time_list!=[]:
            nearest_date = nearestDate(date_time_to_check, date_time_list)
        
            ## Difference between the two in days
            diff_nearest_date = (nearest_date-date_time_to_check).days

            rowIndex = data[data['date'].astype(str).str.contains(str(nearest_date))].index
            data.loc[rowIndex, 'diff_nearest_clean'] = diff_nearest_date
    

In [7]:
data.to_csv('regions_cleaned_with_diff.csv')